<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

In [2]:
# Lab 3.1.3:
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and database maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands".

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [14]:
import sqlite3

In [16]:
import sqlite3

sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [18]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [20]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

In [50]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
cursor = conn.cursor()

try:
    # Check if the table exists and drop it if it does
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='houses';")
    table_exists = cursor.fetchone()
    
    if table_exists:
        cursor.execute('DROP TABLE houses;')
        print("Table 'houses' dropped successfully.")
    else:
        print("Table 'houses' does not exist.")

    # Commit the changes
    conn.commit()

    # Create the 'houses' table
    cursor.execute('''
    CREATE TABLE houses (
        field1 INTEGER PRIMARY KEY,
        sqft INTEGER,
        bdrms INTEGER,
        age INTEGER,
        price INTEGER
    );
    ''')
    print("Table 'houses' created successfully.")
    
    # Commit the changes
    conn.commit()

    # Insert initial data into the 'houses' table
    houses_data = [
        (1, 1500, 3, 10, 300000),
        (2, 2500, 4, 5, 450000),
        (3, 1800, 3, 20, 250000)
    ]
    
    cursor.executemany('''
    INSERT INTO houses (field1, sqft, bdrms, age, price) VALUES (?, ?, ?, ?, ?)
    ''', houses_data)
    
    # Commit the changes
    conn.commit()
    print("Initial data inserted successfully.")

    # Insert the new record
    last_sale = (None, 4000, 5, 22, 619000)
    cursor.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', last_sale)
    
    # Commit the changes
    conn.commit()
    print("New record inserted successfully.")



Table 'houses' dropped successfully.
Table 'houses' created successfully.
Initial data inserted successfully.
New record inserted successfully.


With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [22]:
last_sale = (None, 4000, 5, 22, 619000)
cursor.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', last_sale)
conn.commit()
print("New record inserted successfully.")


NameError: name 'cursor' is not defined

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key.

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [24]:
# A
query = 'Select * from houses'
c.execute(query)
c.fetchall()
#pd.DataFrame(c.fetchall())

[]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more.

In [26]:
import numpy as np
import pandas as pd

In [28]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
csv_file = 'housing-data.csv'

data = genfromtxt('housing-data.csv', dtype = 'i8',
                    delimiter = ',', skip_header = 1).tolist()


In [30]:
csv_file = pd.read_csv('housing-data.csv')


Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [118]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)




*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [120]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [32]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)



ProgrammingError: Incorrect number of bindings supplied. The current statement uses 5, and there are 4 supplied.

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [34]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)


#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [36]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')


QUIZ: What would `DELETE FROM houses` do?

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [ ]:
# ANSWER

**2. The following query calculates the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [38]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(3, 1200.0, 250000.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [88]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database.

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [40]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('../DATA/housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [42]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

47

In [138]:
import sqlite3

# Connect to the SQLite database
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
cursor = conn.cursor()

try:
    # Execute the query
    query = '''
    SELECT sqft, AVG(price) AS average_price
    FROM houses
    GROUP BY sqft;
    '''
    cursor.execute(query)
    
    # Fetch all results
    results = cursor.fetchall()
    
    # Print the results
    for row in results:
        print(f"Size: {row[0]} sqft, Average Price: ${row[1]:,.2f}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()


Size: 1500 sqft, Average Price: $300,000.00
Size: 1800 sqft, Average Price: $250,000.00
Size: 2500 sqft, Average Price: $450,000.00
Size: 4000 sqft, Average Price: $619,000.00


## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate functions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [44]:
# Reading CSV to Dataframe
orders = pd.read_csv('../DATA/P12-ListOfOrders.csv', encoding = 'utf-8')

orders_break_down =  pd.read_csv('../DATA/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [46]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [48]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


In [ ]:
#### 2. Replace Space with Underscore in Column Names

In [50]:
orders.columns = orders.columns.str.replace(' ', '_')
print(orders)

             Order_ID  Order_Date      Customer_Name         Country
0     IT-2011-3647632  2011-01-01       Eugene Moren          Sweden
1     ES-2011-4869686  2011-01-03  Dorothy Dickinson  United Kingdom
2     ES-2011-4939443  2011-01-04     Arthur Prichep          France
3     IT-2011-2942451  2011-01-04     Grant Thornton  United Kingdom
4     ES-2011-3848439  2011-01-05   Michael Granlund          France
...               ...         ...                ...             ...
4112  ES-2014-2815584  2014-12-31        Erica Smith         Germany
4113  ES-2014-3458802  2014-12-31         John Grady  United Kingdom
4114  ES-2014-3638865  2014-12-31    Susan Vittorini  United Kingdom
4115  ES-2014-4785777  2014-12-31      Dennis Pardue         Germany
4116  IT-2014-3715679  2014-12-31           Jim Kriz     Netherlands

[4117 rows x 4 columns]


In [52]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]
print(orders)

             order_id  order_date      customer_name         country
0     IT-2011-3647632  2011-01-01       Eugene Moren          Sweden
1     ES-2011-4869686  2011-01-03  Dorothy Dickinson  United Kingdom
2     ES-2011-4939443  2011-01-04     Arthur Prichep          France
3     IT-2011-2942451  2011-01-04     Grant Thornton  United Kingdom
4     ES-2011-3848439  2011-01-05   Michael Granlund          France
...               ...         ...                ...             ...
4112  ES-2014-2815584  2014-12-31        Erica Smith         Germany
4113  ES-2014-3458802  2014-12-31         John Grady  United Kingdom
4114  ES-2014-3638865  2014-12-31    Susan Vittorini  United Kingdom
4115  ES-2014-4785777  2014-12-31      Dennis Pardue         Germany
4116  IT-2014-3715679  2014-12-31           Jim Kriz     Netherlands

[4117 rows x 4 columns]


In [54]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]
print(orders)

             order_id  order_date      customer_name         country
0     IT-2011-3647632  2011-01-01       Eugene Moren          Sweden
1     ES-2011-4869686  2011-01-03  Dorothy Dickinson  United Kingdom
2     ES-2011-4939443  2011-01-04     Arthur Prichep          France
3     IT-2011-2942451  2011-01-04     Grant Thornton  United Kingdom
4     ES-2011-3848439  2011-01-05   Michael Granlund          France
...               ...         ...                ...             ...
4112  ES-2014-2815584  2014-12-31        Erica Smith         Germany
4113  ES-2014-3458802  2014-12-31         John Grady  United Kingdom
4114  ES-2014-3638865  2014-12-31    Susan Vittorini  United Kingdom
4115  ES-2014-4785777  2014-12-31      Dennis Pardue         Germany
4116  IT-2014-3715679  2014-12-31           Jim Kriz     Netherlands

[4117 rows x 4 columns]


#### 3. Check DataTypes

In [56]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [90]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [115]:
import pandas as pd
import sqlite3

In [127]:
# Establishing Local DB connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

# Save these two dataframes as a table in sqlite


#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [135]:
import sqlite3
import pandas as pd

# Establish the database connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

# Define the SQL query
query = """
    SELECT *
    FROM orders
    LIMIT 10;
"""

# Read the SQL query into a DataFrame
orders_head_10 = pd.read_sql(query, con=db_connection)

# Display the DataFrame
print(orders_head_10)



          order_id  order_date      customer_name         country
0  IT-2011-3647632  2011-01-01       Eugene Moren          Sweden
1  ES-2011-4869686  2011-01-03  Dorothy Dickinson  United Kingdom
2  ES-2011-4939443  2011-01-04     Arthur Prichep          France
3  IT-2011-2942451  2011-01-04     Grant Thornton  United Kingdom
4  ES-2011-3848439  2011-01-05   Michael Granlund          France
5  ES-2011-5433855  2011-01-07       Dave Poirier          France
6  IT-2011-4546695  2011-01-08      Darren Powers          France
7  ES-2011-1138719  2011-01-11       Eric Murdock           Italy
8  ES-2011-1466305  2011-01-11         Mick Brown         Austria
9  ES-2011-4359424  2011-01-11  Dorothy Dickinson           Spain


In [147]:
# Select Number of Distinct Orders for Each Customer
import sqlite3
import pandas as pd

# Establish the database connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

# Define the SQL query
query = """
    SELECT Customer_name, COUNT(DISTINCT Order_ID) AS Num_Orders
    FROM Orders
    GROUP BY Customer_name;
"""

# Execute the query and read the results into a DataFrame
customer_orders_count = pd.read_sql(query, con=db_connection)

# Display the DataFrame
print(customer_orders_count)



          customer_name  Num_Orders
0         Aaron Bergman          11
1         Aaron Hawkins           7
2        Aaron Smayling          12
3       Adam Bellavance           4
4             Adam Hart           8
..                  ...         ...
787        Xylona Preis           2
788       Yana Sorensen           5
789      Yoseph Carroll          11
790    Zuschuss Carroll           9
791  Zuschuss Donatelli           6

[792 rows x 2 columns]


#### 6. Select Number of Customers for Each Country

In [181]:
import sqlite3

# Establish the database connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

# Get list of all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(tables_query, con=db_connection)
print("Tables in the database:")
print(tables)

# Check columns of the Customers table
columns_query = "PRAGMA table_info(Customers);"
columns = pd.read_sql(columns_query, con=db_connection)
print("Columns in the Customers table:")
print(columns)




Tables in the database:
                name
0             orders
1  orders_break_down
Columns in the Customers table:
Empty DataFrame
Columns: [cid, name, type, notnull, dflt_value, pk]
Index: []


In [185]:
import sqlite3
import pandas as pd

# Establish the database connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

# Define the SQL query
query = """
    SELECT Country, COUNT(*) AS Num_Customers
    FROM Customers
    GROUP BY Country;
"""

# Execute the query and read the results into a DataFrame
try:
    country_customers_count = pd.read_sql(query, con=db_connection)
    # Display the DataFrame
    print(country_customers_count)
except Exception as e:
    print(f"An error occurred: {e}")




An error occurred: Execution failed on sql '
    SELECT Country, COUNT(*) AS Num_Customers
    FROM Customers
    GROUP BY Country;
': no such table: Customers


In [187]:
import sqlite3
import pandas as pd

# Establish the database connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

query = """
    SELECT Country, COUNT(*) AS Num_Customers
    FROM customer_name
    GROUP BY Country;
"""

# Execute the query and read the results into a DataFrame
country_customers_count = pd.read_sql(query, con=db_connection)

# Display the DataFrame
print(country_customers_count)




DatabaseError: Execution failed on sql '
    SELECT Country, COUNT(*) AS Num_Customers
    FROM customer_name
    GROUP BY Country;
': no such table: customer_name

#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [189]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
import sqlite3
import pandas as pd

# Establish the database connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

# Define the SQL query
query = """
    SELECT discount, sales, quantity
    FROM orders_break_down;
"""

# Execute the query and read the results into a DataFrame
try:
    order_details = pd.read_sql(query, con=db_connection)
    # Display the DataFrame
    print(order_details)
except Exception as e:
    print(f"An error occurred: {e}")

 


      discount  sales  quantity
0          0.5   45.0         3
1          0.0  854.0         7
2          0.0  140.0         3
3          0.5   27.0         2
4          0.5   17.0         2
...        ...    ...       ...
8042       0.0  245.0         2
8043       0.5   30.0         2
8044       0.5   23.0         4
8045       0.5  108.0         3
8046       0.0  867.0         5

[8047 rows x 3 columns]


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [195]:
# Select discount, sales, quantity, total price for Each Order Id from orders_break_down Table
import sqlite3
import pandas as pd

# Establish the database connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

# Define the SQL query
query = """
    SELECT Order_ID, discount, sales, quantity, total_price
    FROM orders_break_down;
"""

# Execute the query and read the results into a DataFrame
try:
    order_details = pd.read_sql(query, con=db_connection)
    # Display the DataFrame
    print(order_details)
except Exception as e:
    print(f"An error occurred: {e}")



An error occurred: Execution failed on sql '
    SELECT Order_ID, discount, sales, quantity, total_price
    FROM orders_break_down;
': no such column: total_price


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [197]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
import sqlite3
import pandas as pd

# Establish the database connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

# Define the SQL query
query = """
    SELECT *
    FROM orders_break_down
    WHERE total_price > 100;
"""

# Execute the query and read the results into a DataFrame
try:
    orders_above_100 = pd.read_sql(query, con=db_connection)
    # Display the DataFrame
    print(orders_above_100)
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: Execution failed on sql '
    SELECT *
    FROM orders_break_down
    WHERE total_price > 100;
': no such column: total_price


#### 8. Select All Customers And The Products They Have Bought

In [199]:
# Select All Order Ids, Customers And The Products (product_name) They Have Bought
import sqlite3
import pandas as pd

# Establish the database connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

# Define the SQL query
query = """
    SELECT 
        orders.Order_ID,
        customers.Customer_Name,
        products.Product_Name
    FROM 
        orders
    JOIN 
        customers ON orders.Customer_ID = customers.Customer_ID
    JOIN 
        order_details ON orders.Order_ID = order_details.Order_ID
    JOIN 
        products ON order_details.Product_ID = products.Product_ID;
"""

# Execute the query and read the results into a DataFrame
try:
    order_customer_product = pd.read_sql(query, con=db_connection)
    # Display the DataFrame
    print(order_customer_product)
except Exception as e:
    print(f"An error occurred: {e}")



An error occurred: Execution failed on sql '
    SELECT 
        orders.Order_ID,
        customers.Customer_Name,
        products.Product_Name
    FROM 
        orders
    JOIN 
        customers ON orders.Customer_ID = customers.Customer_ID
    JOIN 
        order_details ON orders.Order_ID = order_details.Order_ID
    JOIN 
        products ON order_details.Product_ID = products.Product_ID;
': no such table: customers


#### 9.A Select Number of 'Furniture' Orders For Each Country

In [201]:
# Select Number of 'Furniture' Orders For Each Country
import sqlite3
import pandas as pd

# Establish the database connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

# Define the SQL query
query = """
    SELECT 
        orders.Country,
        COUNT(orders.Order_ID) AS Num_Furniture_Orders
    FROM 
        orders
    JOIN 
        order_details ON orders.Order_ID = order_details.Order_ID
    JOIN 
        products ON order_details.Product_ID = products.Product_ID
    WHERE 
        products.Category = 'Furniture'
    GROUP BY 
        orders.Country;
"""

# Execute the query and read the results into a DataFrame
try:
    furniture_orders_by_country = pd.read_sql(query, con=db_connection)
    # Display the DataFrame
    print(furniture_orders_by_country)
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: Execution failed on sql '
    SELECT 
        orders.Country,
        COUNT(orders.Order_ID) AS Num_Furniture_Orders
    FROM 
        orders
    JOIN 
        order_details ON orders.Order_ID = order_details.Order_ID
    JOIN 
        products ON order_details.Product_ID = products.Product_ID
    WHERE 
        products.Category = 'Furniture'
    GROUP BY 
        orders.Country;
': no such table: order_details


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [203]:
# Select Number of 'Furniture' Orders For The Country Denmark
import sqlite3
import pandas as pd

# Establish the database connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

# Define the SQL query
query = """
    SELECT 
        COUNT(orders.Order_ID) AS Num_Furniture_Orders
    FROM 
        orders
    JOIN 
        order_details ON orders.Order_ID = order_details.Order_ID
    JOIN 
        products ON order_details.Product_ID = products.Product_ID
    WHERE 
        products.Category = 'Furniture'
        AND orders.Country = 'Denmark';
"""

# Execute the query and read the results into a DataFrame
try:
    furniture_orders_denmark = pd.read_sql(query, con=db_connection)
    # Display the DataFrame
    print(furniture_orders_denmark)
except Exception as e:
    print(f"An error occurred: {e}")

# Close the database connection
db_connection.close()


An error occurred: Execution failed on sql '
    SELECT 
        COUNT(orders.Order_ID) AS Num_Furniture_Orders
    FROM 
        orders
    JOIN 
        order_details ON orders.Order_ID = order_details.Order_ID
    JOIN 
        products ON order_details.Product_ID = products.Product_ID
    WHERE 
        products.Category = 'Furniture'
        AND orders.Country = 'Denmark';
': no such table: order_details


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [ ]:
# Select Total Sales With Discount (discount > 0) and Without Discount (discount = 0) for Each Country
Cannot read tables

#### 11.A Select Total Quantity, Total Sales for Each Country

In [ ]:
# Select Total Quantity, Total Sales for Each Country

#### 11.B Select Top 3 Countries and Quantities Based on Total Sales

In [ ]:
# Select Top 3 Countries and Quantities Based on Total Sales

#### 11.C Select Bottom 3 Countries and Sales Based On Total Quantities

In [ ]:
# Select Bottom 3 Countries and Sales Based On Total Quantities

#### 12. Select Average Sales By Category For The Country 'France'

In [ ]:
# Select Average Sales By Category For The Country 'France'

#### 13. Select Country, Category and Average Sales Where Average Sales is The Highest

In [ ]:
# Select Country, Category and Average Sales Where Average Sales is The Highest

### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams.
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  



---



---



> > > > > > > > > © 2023 Institute of Data


---



---



